In [ ]:
pip install azure-storage-blob


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 6.0 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
def extraction(url,fname):
  headers = {
      'X-Api-Key':'ApiKeyValue'
  }

  response = requests.get(url, headers=headers)

  if response.status_code == 200:
      try:

          df = pd.read_csv(StringIO(response.text))
      except Exception as e:
          print("Error: Unable to parse CSV")
          print(e)
  else:
      print(f"Error: {response.status_code}")
      print(response.text)
  csv_file = fname+'.csv'
  if csv_file=='PoliceStation.csv':
    columns=['gis_id','srcid_t','edit_date','subtype','address','city','state','zipcode','x_coord','y_coord','name','nghbrhd','commander','phone']
    df=df[columns]
  else:
    columns=[	'IncidentNumber' , 'ArrestNumber' ,	'Age' , 'Gender' , 'Race' , 'ArrestDateTime' ,	'ArrestLocation' ,	'IncidentOffence' , 'IncidentLocation' , 'Charge' , 'ChargeDescription' , 'District' , 'Post' , 'Neighborhood' , 'Latitude' , 'Longitude']
    df=df[columns]
    df['IncidentNumber'].fillna(0, inplace=True)
    df['ArrestNumber'].fillna(0, inplace=True)
    df = df[df['ArrestNumber'] != 0]
    df = df.drop_duplicates(subset=['ArrestNumber'], keep='first')


  connection_string = 'DefaultEndpointsProtocol=https;AccountName=crimeanalytics;AccountKey=RErlJSqJJjgTig9G+2RowOIxQcHyp8qg2AGAuK97nJr7DMuRvPz53o8/p/PDBDsUesSTgjZ0dVuZ+AStFn1KqQ==;EndpointSuffix=core.windows.net'
  container_name = 'crime-analytics-container'
  df.to_csv(csv_file, index=False)
  blob_service_client = BlobServiceClient.from_connection_string(connection_string)

  blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_file)


  with open(csv_file, 'rb') as data:
      blob_client.upload_blob(data, overwrite=True)

  print(f"CSV file '{csv_file}' uploaded to Azure Blob Storage in container '{container_name}'")

In [ ]:
dict={'PoliceStation':'https://data.baltimorecity.gov/api/download/v1/items/e734ec5b2f994f3291c9226259e16525/csv?layers=0',
      'BPDArrests':'https://data.baltimorecity.gov/api/download/v1/items/619ec10c14b346f784a5a07bad4c43cd/csv?layers=0'}


In [ ]:
for key, value in dict.items():
    extraction(value, key)

CSV file 'PoliceStation.csv' uploaded to Azure Blob Storage in container 'crime-analytics-container'


<ipython-input-3-620d0346f050>:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['IncidentNumber'].fillna(0, inplace=True)
<ipython-input-3-620d0346f050>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

CSV file 'BPDArrests.csv' uploaded to Azure Blob Storage in container 'crime-analytics-container'
